In [ ]:
# !conda install -c intel mkl --yes

# !conda remove mkl mkl-service --yes
# !conda install nomkl numpy scipy scikit-learn numexpr --yes


***카테고리 별로 고유 넘버를 가져오는 크롤러***

In [3]:
import json
import requests
import pandas as pd
import numpy as np
import gc

def crawl_category_nums():
    """
    category dictionary function 
    카테고리 번호만으로 json 파일 만들기 
    번호만 있었을 때 나중에 상세정보 크롤링을 할 때 편리할 것으로 예상됨. 
    
    만약 번호말고 다른 정보도 필요하다면 코드를 수정하여 
    카테고리 이름도 가져올 수 있음. 
    """
    
    base_url = 'https://api.bunjang.co.kr/api/1/categories/list.json' # 비동기 방식으로 크롤링할 때 쓰일 base url
    headers = { 
    'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
        }
    res = requests.get(base_url, headers = headers)
    data = res.json()
    
    cat_1, cat_2, cat_3 = None, None, None # 변수 초기화
    cat_dict = {} # 최종적으로 넣어줄 category dictionary 
    
    for cats1 in data['categories']:
        cat_1 = cats1['id'] # 가장 상위 카테고리 cat_1
        try: # '기타' 카테고리 이후로는 'categories'가 없음.  
            tmp_dict = {} # 임시적으로 담아줄 dictionary
            for cats2 in cats1['categories']: 
                cat_2 = cats2['id'] # 다음 카테고리인 cat_2
                tmp_dict[cat_2] = [] # cat_2 이름별로 따로 빈 리스트 만들어주기 
                try: # cat_1, cat_2 다음 cat_3가 없는 경우가 있음. 
                    for cats3 in cats2['categories']: # 다음 세부 항목 (cat_3)는 'categories' 안에 들어있음. 
                        cat_3 = cats3['id'] 
                        tmp_dict[cat_2].append(cat_3) # 아까 만들어 주었던 빈 리스트에 cat_3 항목들 넣어주기 
                    cat_dict[cat_1] = tmp_dict
                except:
                    cat_3 = None # cat_3가 없는 경우 None으로 채워주기
                    tmp_dict[cat_2].append(cat_3)
                cat_dict[cat_1] = tmp_dict

        except: # '기타' 카테고리에 걸칠 경우 break
            break 
            
    json_data = json.dumps(cat_dict, indent=4)  
    with open('bungae_unique_category_numbers.json', 'w') as f: # json 파일로 내보내기 
        f.write(json_data)
        
    gc.collect() 
        
    return

if __name__ == '__main__':
    crawl_category_nums()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


***카테고리 번호에 따라 이름을 알 수 있는 json 파일 만드는 함수***

In [ ]:
# 생각해보니 카테고리 번호를 통해 카테고리 이름을 알고 싶은 경우가
# 무조건 생길 것으로 예상됨
# 그래서 카테고리 번호를 넣으면 카테고리 이름을 알려주는 크롤러 혹은 함수를 만들고자 함. 
# 나중에 이전 함수와 같이 활용되어 카테고리 분류에 사용될 것으로 예상됨. 

import json
import requests
import pandas as pd
import numpy as np
from collections import Counter

def mapping_category_titles_and_nums() -> None:
    """
    카테고리 번호를 통해 카테고리 이름을 알 수 있도록 해주는 함수. 
    
    해당 함수는 단순히 카테고리 번호에 따라 카테고리 이름이 무엇인지 매핑하여
    json파일로 내보내는 함수입니다. 
    """
    
    base_url = 'https://api.bunjang.co.kr/api/1/categories/list.json' # 비동기 방식으로 크롤링할 때 쓰일 base url
    headers = { 
    'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
        }
    res = requests.get(base_url, headers = headers)
    data = res.json()
    
    cat_1, cat_2, cat_3 = None, None, None # 변수 초기화
    title_1, title_2, title_3 = None, None, None
    cat_1_dict, cat_2_dict, cat_3_dict = {}, {}, {} # 카테고리 hierachy 대로 따로 ditionary 생성
        
    for cats1 in data['categories']:
        cat_1 = cats1['id'] # 가장 상위 카테고리 cat_1
        title_1 = cats1['title'] # 가장 상위 카테고리 이름
        cat_1_dict[cat_1] = title_1 # cat_1_dict에 넣어주기 
        try: # '기타' 카테고리 이후로는 'categories'가 없음.  
            for cats2 in cats1['categories']: 
                cat_2 = cats2['id'] # 다음 카테고리인 cat_2
                title_2 = cats2['title'] # 다음 카테고리 이름
                cat_2_dict[cat_2] = title_2 # cat_2_dict에 넣어주기 
                try: # cat_1, cat_2 다음 cat_3가 없는 경우가 있음. 
                    for cats3 in cats2['categories']: # 다음 세부 항목 (cat_3)는 'categories' 안에 들어있음. 
                        cat_3 = cats3['id'] # 다음 카테고리인 cat_3
                        title_3 = cats3['title'] # 카테고리명
                        cat_3_dict[cat_3] = title_3 # cat_3_dict에 넣어주기 
                except: # cat_3가 없는 경우
                    continue 

        except: # '기타' 카테고리에 걸칠 경우 break
            break 
        
        final_dict = dict(cat_1_dict, **cat_2_dict, **cat_3_dict)
        json_data = json.dumps(final_dict, indent=4)  
        with open('bungae_mapping_category_nums_and_titles.json', 'w') as f: # json 파일로 내보내기 
            f.write(json_data)
        
    return 

***상품 상세 정보 크롤러***

In [ ]:
# multiprocessing 활용
# 위에서 카테고리 별로 고유 번호를 나눈 이유는 
# 카테고리 별로 병렬적으로 크롤링을 해보기 위함. 

In [247]:
import json
import requests
import pandas as pd
import numpy as np
from multiprocessing import Pool
import multiprocessing
import time
import gc
import os
import random


############### 첫번째 pool ########################################################

def extract_usable_cat_id_only():
    # 이전에 만들었던 json 파일에서 정보 가져오기
    with open('./bungae_unique_category_numbers.json', 'r') as file: 
        data = json.load(file)
    
    # 최상단 목록을 뺀 나머지 cat_2, cat_3 카테고리 번호들 가져오기 
    new_dict = dict()
    for i in list(data.values()):
        new_dict.update(i)
        
    # 3번째 하위 카테고리 목록이 없는 경우도 있는데
    # 그럴때는 2번째 하위 항목의 카테고리 번호로 크롤링을 해야함. 
    # 때문에 크롤링할 때 필요한 None 값이 아닌 카테고리 번호들만 따로 리스트로 만들기
    usable_cat_list = list()
    for x in new_dict.keys():
        if new_dict[x][0] == None:
            usable_cat_list.append(x)
        else:
            usable_cat_list.extend(new_dict[x])
            
    return usable_cat_list

def prd_id_extractor(category_num):
    """
    category id를 사용하여 상품 고유 번호를 뽑아오는 크롤러 입니다. 
    """
    page_num = 0
    while True: 
        url = "https://api.bunjang.co.kr/api/1/find_v2.json?"
        params = {
            'f_category_id' : category_num, # 바뀔 부분 1
            'page' : str(page_num), # 바뀔 부분 2 
            'order': 'date', # 최신순? 
            'req_ref': 'category', 
            'request_id': '2023517001018', # 이 부분 어떻게 해결? 
            'stat_device': "w",
            "n" : '100',
            'version' : '4'
        }
        headers = {
            'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
        }
        res = requests.get(url, headers = headers, params=params)
        data = res.json()

        prd_id_list = [datas['pid'] for datas in data['list']] # product id 리스트로 만들어주기 

        if page_num == 4: # 우선 page수를 4까지 지정 ==> 대략 400개 
            break
            
        if data['no_result']: # page수가 지정한 페이지 수보다 적을 경우 break
            break
            
        page_num += 1
        
    print(f"{category_num} done")
    time.sleep(random.randint(10,15)) # 한 카테고리에서의 크롤링 끝나고 잠시 대기하고 다음 카테고리 크롤링 할 수 있도록
                                     # block의 가능성을 줄여주기 위해서 
    return prd_id_list


def prd_mp_1(cat_list:list):
    """
    첫번째 pool 함수
    카테고리 리스트를 받아서 전체 상품 번호 리스트를 반환하는 함수입니다. 
    """
    pool = multiprocessing.Pool(os.cpu_count()) # 사용 가능한 cpu 확인
    result = pool.map(prd_id_extractor, cat_list) # 카테고리 리스트에 따른 모든 상품 번호 가져오도록 Pool
    
    prd_list = list()
    for r in result:
        prd_list += r
        
    pool.close()
    pool.join()
    
    gc.collect()
    return prd_list # 모든 상품 id가 들어있는 리스트 

############### 두번째 pool ########################################################
def prd_crawler(prd_id): 
    """
    상품 번호에 따라 상품 정보를 가져오는 크롤러입니다. 
    """
    url = 'https://api.bunjang.co.kr/api/pms/v1/products-detail/{}?viewerUid=-1'.format(prd_id)
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers = headers)
    data = res.json()
    
    try:
        product_id = data['data']['product']['pid']
        product_name = data['data']['product']['name']
        image_url = data['data']['product']['imageUrl']
        image_cnt = data['data']['product']['imageCount']
        price = data['data']['product']['price']
        info = data['data']['product']['description']
        date = data['data']['product']['updatedAt']
        cat_id = data['data']['product']['category']['id']
    except:
        product_id, product_name, image_url, image_cnt, price, info, cat_id, date =\
            None, None, None, None, None, None, None, None
        
    time.sleep(random.randint(30,60)) # block을 당할 가능성을 줄이기 위해 time.sleep 조금 길게 둠. 
    
    return [product_id, product_name, image_url, image_cnt, price, info, cat_id, date] # 리스트로 반환

def prd_mp_2(prd_list:list):
    """
    두번째 pool 함수
    """
    pool = multiprocessing.Pool(os.cpu_count()) # 사용 가능한 cpu 확인
    product_info_list = pool.map(prd_crawler, prd_list) # 카테고리 리스트에 따른 모든 상품 번호 가져오도록 Pool
                                                # --> 바로 리스트로 나옴
    pool.close()
    pool.join()
    
    gc.collect()
    
    prd_df = pd.DataFrame(product_info_list, columns = ['product_id', 'product_name', 'image_url', 'image_cnt', 'price', 'info', 'cat_id', 'date'])
    prd_df = prd_df.dropna()
    
    return prd_df # 모든 상품 id가 들어있는 리스트 

if __name__ == '__main__':
    start = time.time()
    cat_list = extract_usable_cat_id_only() # url과 같이 사용이 가능한 카테고리 리스트
    prd_list = prd_mp_1(cat_list) # Pool 사용하여 카테고리에 따라 모든 상품 id 가져오기 
    prd_df = prd_mp_2(prd_list) # 다음 Pool
    prd_df.to_csv('test_df.csv', index=False) # 데이터 프레임으로 만들어주기
        
    #### 필요한 후속처리 ####
    # cat_id --> cat_1, cat_2, cat_3로 나눌 것인지?
    # imageUrl + imageCount 해서 image_list로 다시 만들 것인지?
    
    end = time.time()
    print('수행시간', end-start)
    gc.collect()
    

In [ ]:
!python tmp_bungae_crawler.py

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


700120200 done
600200008 done
750200100 done
750610500 done
600100007 done
910200008 done
600700005 done
700120100 done
600200007 done
700350100 done
600100005 done
750200200 done
750800100 done
600700002 done
700160 done
910200006 done
600200009 done
700350500 done
600100004 done
750200300 done
750800200 done
600700003 done
700140 done
700350300 done
910200007 done
600100999 done
600200012 done
750810100 done
750200500 done
910500004 done
600700007 done
750810300 done
700170 done
700350400 done
600600001 done
600200010 done
750200600 done
600700999 done
910500001 done
750810200 done
700600300 done
700130200 done
750200700 done
700350600 done
600600002 done
600400003 done
750810999 done
910500003 done
700600500 done
750200400 done
700350700 done
700130100 done
600600003 done
600400002 done
750910100 done
910500008 done
700600400 done
750200800 done
700180 done
700350950 done
600400001 done
600600005 done
750910200 done
700600900 done
700900300 done
750100100 done
910500002 done
700500 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
product id 152973699 done
product id 113552564 done
product id 173363635 done
product id 132901763 done
product id 223570589 done
product id 76885833 done
product id 217810796 done
product id 179565516 done
product id 192449588 done
product id 212647231 done
product id 174405088 done
product id 217861132 done
product id 202200186 done
product id 214726638 done
product id 218330744 done
product id 156236866 done
product id 168848866 done
product id 61554535 done
product id 205885003 done
product 

product id 214670624 done
product id 218341287 done
product id 224089816 done
product id 224173773 done
product id 180956695 done
product id 221310618 done
product id 183413784 done
product id 122563829 done
product id 224335066 done
product id 224193659 done
product id 209359652 done
product id 223672809 done
product id 215229508 done
product id 183844033 done
product id 201003263 done
product id 220840495 done
product id 220924604 done
product id 172324242 done
product id 220152647 done
product id 220268712 done
product id 79016097 done
product id 224336475 done
product id 135833308 done
product id 191196478 done
product id 224322464 done
product id 193122661 done
product id 181022291 done
product id 192098955 done
product id 220784541 done
product id 216794633 done
product id 106473861 done
product id 223339477 done
product id 224334867 done
product id 178118554 done
product id 211844329 done
product id 219199870 done
product id 217701250 done
product id 220396270 done
product id 22

product id 223335960 done
product id 197483521 done
product id 218567642 done
product id 187248438 done
product id 213150340 done
product id 214986297 done
product id 212389938 done
product id 219990872 done
product id 208977084 done
product id 224291586 done
product id 187217772 done
product id 210885075 done
product id 213150228 done
product id 220455014 done
product id 112429659 done
product id 219157270 done
product id 213162436 done
product id 221595949 done
product id 199718037 done
product id 224283493 done
product id 159288636 done
product id 199784317 done
product id 218455201 done
product id 211030073 done
product id 199192932 done
product id 206046827 done
product id 221592028 done
product id 218431825 done
product id 212211678 done
product id 221288877 done
product id 221401108 done
product id 162473108 done
product id 219735144 done
product id 200234716 done
product id 218782667 done
product id 204590850 done
product id 165172213 done
product id 182267265 done
product id 2